In [8]:
from run_crypto import fetch_binance_data
from datetime import datetime, timedelta
import time

symbol = 'BTCUSDT'
interval = '1m'
current_time_ms = int(time.time() * 1000)
twelve_hours_ms = 12 * 60 * 60 * 1000
start_time = current_time_ms - twelve_hours_ms

# Fetch data
df = fetch_binance_data(symbol, interval, start_time)
# df = fetch_coinbase_data(product_id=product, start_time=start_timestamp, end_time=end_timestamp, granularity='60')
print(df.head())

       timestamp            open            high             low  \
0  1732686060000  92514.47000000  92514.47000000  92418.48000000   
1  1732686120000  92418.48000000  92418.48000000  92418.48000000   
2  1732686180000  92712.77000000  92734.98000000  92712.77000000   
3  1732686240000  92734.98000000  92734.98000000  92734.98000000   
4  1732686300000  92734.98000000  92734.98000000  92734.98000000   

            close      volume     close_time quote_asset_volume  \
0  92418.48000000  0.00135000  1732686119999       124.84016090   
1  92418.48000000  0.00000000  1732686179999         0.00000000   
2  92734.98000000  0.00096000  1732686239999        89.01758520   
3  92734.98000000  0.00010000  1732686299999         9.27349800   
4  92734.98000000  0.01309000  1732686359999      1213.90088820   

   number_of_trades taker_buy_base_asset_volume taker_buy_quote_asset_volume  \
0                 4                  0.00000000                   0.00000000   
1                 0         

In [10]:
# Importar las funciones necesarias del script original
from run_crypto import calculate_technical_indicators,fetch_binance_data, fetch_and_process_data, train_predictive_model, prepare_model_and_scaler
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Cargar los datos históricos del archivo
symbol = 'BTCUSDT'
interval = '1m'
current_time_ms = int(time.time() * 1000)
twelve_hours_ms = 12 * 60 * 60 * 1000
start_time = current_time_ms - twelve_hours_ms

# Fetch data
prices_df = fetch_binance_data(symbol, interval, start_time)

# Asegurarse de que el timestamp sea un datetime
# prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'])

# Calcular indicadores técnicos
prices_df = calculate_technical_indicators(prices_df)

# Agregar la columna de precios futuros (15 minutos después)
prices_df['future_price'] = prices_df['close'].shift(-15)

# Preparar datos y modelo
features = ['rsi', 'sma', 'ema', 'macd', 'macd_signal']
target = 'close'
model, scaler = train_predictive_model(prices_df, features, target)

# Realizar el backtesting
results = []
for i in range(len(prices_df) - 15):
    current_price = prices_df['close'].iloc[i]
    future_price = prices_df['future_price'].iloc[i]

    # Seleccionar las características de entrada
    input_data = prices_df[features].iloc[i:i+1]
    input_data_scaled = scaler.transform(input_data)

    # Hacer la predicción
    predicted_price = model.predict(input_data_scaled)[0]

    # Decidir y evaluar el resultado
    decision = "BUY" if predicted_price >= current_price else "SELL"
    result = "CORRECT" if (decision == "BUY" and predicted_price >= future_price) or \
                          (decision == "SELL" and predicted_price < future_price) else "WRONG"
    
    # Agregar a los resultados
    results.append({
        "timestamp": prices_df['timestamp'].iloc[i],
        "current_price": current_price,
        "predicted_price": predicted_price,
        "future_price": future_price,
        "decision": decision,
        "result": result,
    })

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results)

# Mostrar los resultados
from IPython.display import display
display(results_df.head(20))

# Guardar los resultados en un archivo CSV
results_df.to_csv('/mnt/data/backtesting_results.csv', index=False)

print("Backtesting completado. Resultados guardados en 'backtesting_results.csv'.")

,timestamp,current_price,predicted_price,future_price,decision,result
0,1732687320000,92734.99,92742.241861,92797.53,BUY,WRONG
1,1732687380000,92734.98,92742.241861,92852.95,BUY,WRONG
2,1732687440000,92548.71,92726.127379,93000.00,BUY,WRONG
3,1732687500000,92548.71,92684.932832,93000.00,BUY,WRONG
4,1732687560000,92734.98,92731.407127,93044.10,SELL,CORRECT
5,1732687620000,92735.00,92733.773979,93061.40,SELL,CORRECT
6,1732687680000,92735.00,92736.828561,93061.40,BUY,WRONG
7,1732687740000,92735.00,92736.828561,93069.10,BUY,WRONG
8,1732687800000,92735.00,92736.828561,93121.07,BUY,WRONG
9,1732687860000,92735.00,92739.236361,93148.36,BUY,WRONG


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [3]:
# Importar las funciones necesarias del script existente
from run_crypto import calculate_technical_indicators, simulate_prediction
import pandas as pd
import numpy as np

# Cargar los datos de precios
prices_df = pd.read_csv('/mnt/data/prices.log')

# Convertir la columna de timestamp a datetime
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'])

# Calcular indicadores técnicos
prices_df = calculate_technical_indicators(prices_df)

# Agregar la columna de precios futuros (15 minutos después)
prices_df['future_price'] = prices_df['close'].shift(-15)

# Simular predicciones
prices_df['predicted_price'] = prices_df['close'].apply(simulate_prediction)

# Realizar el backtesting
results = []
for index, row in prices_df[:-15].iterrows():
    timestamp = row['timestamp']
    current_price = row['close']
    predicted_price = row['predicted_price']
    future_price = row['future_price']
    
    decision = "BUY" if predicted_price >= current_price else "SELL"
    result = "CORRECT" if (decision == "BUY" and predicted_price >= future_price) or \
                          (decision == "SELL" and predicted_price < future_price) else "WRONG"
    reason = f"Predicted price ({predicted_price:.2f}) {'>=' if predicted_price >= current_price else '<'} current price ({current_price:.2f}). Future price = {future_price:.2f}."
    
    results.append({
        "timestamp": timestamp,
        "current_price": current_price,
        "predicted_price": predicted_price,
        "future_price": future_price,
        "decision": decision,
        "result": result,
        "reason": reason
    })

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

# Mostrar los primeros resultados
results_df.head()

# Guardar los resultados en un archivo CSV
results_df.to_csv('/mnt/data/backtesting_results.csv', index=False)

# Visualización en Jupyter Notebook
from IPython.display import display
display(results_df.head(20))  # Mostrar las primeras 20 filas

ImportError: cannot import name 'simulate_prediction' from 'run_crypto' (/Users/obarrientos/Documents/Prodigies/apps/crypto/run_crypto.py)